In [1]:
import numpy as np
from numpy.linalg import norm,inv
from scipy.misc import derivative
from numdifftools import Gradient,Hessian
import numdifftools as nd
from scipy.optimize import linprog,minimize_scalar

# 1 - Linear search :


In [2]:
def bissection(f,a,b,tol):
    a,b = min(a,b), max(a,b)
    try :
        while (abs(b-a)>tol):
            c = (a + b) / 2
            if (derivative(f,c) == 0 and derivative(f,c, n=2) > 0):
                return c
            elif (derivative(f, c) <= 0):
                a = c
            else:
                b = c
        return [a,b]
    except:
        print("Erreur : Verifier les bornes")

In [3]:
def fausse_position(f,x0,x1,tol):
    if(x0 == x1):
        print("Erreur : x0 = x1 ! ce n'est pas une intervalle. ")
    x_bef = x0
    x_prev = x1
    while ((abs(x_prev-x_bef)>=tol) or (abs(derivative(f,x_prev))>= tol)):
        d = - derivative(f,x_prev) * ((x_bef - x_prev)/(derivative(f,x_bef)-derivative(f,x_prev)))
        x_bef = x_prev
        x_prev = x_bef + d
    return x_prev    

In [4]:
def raphsonNewton ( fct , x0 , tol ):
    x_k = x0
    if (derivative(fct , x_k,n = 2) != 0):
            d = -(derivative(fct , x_k,n = 1) / derivative(fct , x_k,n = 2))
            x_kk = x_k
            x_k = x_kk + d
    else :
            print("Dérivée seconde de f est nulle!! Newton est inutile dans ce cas")
            return x_k
    while (abs(x_kk - x_k)> tol or abs(derivative(fct , x_k,n = 1)) > tol ):
        if (derivative(fct , x_k,n = 2) != 0):
            d = -(derivative(fct , x_k,n = 1) / derivative(fct , x_k,n = 2))
            x_kk = x_k
            x_k = x_kk + d
        else :
            print("Dérivée seconde de f est nulle!! Newton est inutile dans ce cas")
            return x_k
    return x_k

In [5]:
phi = lambda x: x**3 + 3*x**2 -1
bissection(phi,1,-1,0.001)
raphsonNewton(phi,0.5,0.001)

-0.18350341368301804

In [6]:
def fuc(x):
    return x**4 + 2*(x**2) +1
fausse_position(fuc,0.4,-0.2,0.01)

-0.00015345018797644286

In [7]:
#pas approche 
def armijo(phi,eps = 0.5,alpha0 = 1):
    alpha = alpha0
    while (phi(alpha) <= phi(0) + eps* alpha * derivative(phi, 0)):
        alpha = 2 * alpha
    while (phi(alpha) > phi(0) + eps *alpha* derivative(phi, 0)):
        alpha = alpha / 2
    return alpha
def goldstein(phi,eps = 0.1,a0 = 0,alpha0 = 1 , t = 2):
    alpha = alpha0
    a = a0
    b_inf = 1000
    b = b_inf
    while (True):
        if(phi(alpha) <= (phi(0) + eps *alpha*derivative(phi,0))):
            if(phi(alpha) >= (phi(0)+(1-eps)*alpha)*derivative(phi,0)):
                return alpha
            else :
                a = alpha
            if b < b_inf :
                print("hi")
                alpha = (a+b)/2
            else  : #b >= b_inf:
                alpha = t*alpha
        else :
            b = alpha
            alpha = (a+b)/2
    


In [8]:
print(armijo(fuc),goldstein(fuc))

7.450580596923828e-09 7.450580596923828e-09


# 2 - Constrained optimization

## 2.1 - Gradient descent Algorithm :

In [9]:
def gradient_descent(f,x0,tol):
    x_prev = x0
    x_next = x0
    while(True):
        phi = lambda a : f(x_prev-a*Gradient(f)(x_prev))
        alpha = armijo(phi)
        x_next = x_prev - alpha * Gradient(f)(x_prev)
        if(norm(Gradient(f)(x_next))>tol or norm(x_next-x_prev)>tol ):
            break
    return x_next

In [10]:
def f(x):
    return 0.5*(x[0]**2 +x[1]**2)
x0 = np.array([2,1],dtype = float)
print(gradient_descent(f,x0,0.01))

[4.88498131e-15 2.22044605e-16]


## 2.2 - Newton Algorithm

In [11]:
def is_pos_def(H):
    return np.all(np.linalg.eigvals(H) > 0)

def newton(f, x0, tol):
    x = x0
    d = np.dot(inv(Hessian(f)(x)), Gradient(f)(x))
    while norm(d) > tol:
        phi = lambda alpha: f(x - alpha * d)
        alpha = max(0, raphsonNewton(phi, 1, tol))
        x = x - alpha * d
        print(x)
        d = np.dot(inv(Hessian(f)(x)), Gradient(f)(x))
    return x
def Newton_alg(f , x0 , tol):
    x = x0
    eps = 0.001
    d = np.dot(inv(Hessian(f)(x)) , Gradient(f)(x))
    while (norm(d)>tol):
        phi = lambda alpha : f(x - Gradient(f)(x) * alpha)
        alpha = max(0,armijo(phi))
        x = x - alpha * d
        if (is_pos_def(Hessian(f)(x))):
            d =  inv(Hessian(f)(x)) @ Gradient(f)(x)
        else: 
            d =  inv(eps * np.eye(len(x0))+Hessian(f)(x)) @ Gradient(f)(x)
    return x


In [12]:
x0 = np.array([2,1])
print(Newton_alg(f , x0 , 0.01))
    

[7.54951657e-15 1.99840144e-15]


# 3 - Unconstrained optimization

## 3.1 - Feasible Directions algorithm (directions réalisables)

In [15]:
def feasible_directions(f,A,b,x0):
    x = x0
    while(True):
        # calculate the direction #
        Ik = []
        for i in range(0,len(A)):
            if abs(np.vdot(A[i,:],x) - b[i] )< 0.0001 :
                Ik.append(i)
        #print("Ik :",Ik)
        c = Gradient(f)(x)
        if len(Ik) == 0 :
            d = linprog(c,bounds=(-1,1)).x
        else :
            A_prog = A[Ik,:]
            b_prog = np.zeros(len(Ik))
            d = linprog(c,A_ub = A_prog,b_ub = b_prog,bounds=(-1,1)).x
        #print("direction ",d)
        # calculate the learning rate  alpha_bar - alpha #
        alphas_set = []
        for i in range(0,len(A)):
            if (np.vdot(A[i,:],d)>0) and (i not in Ik) :
                alph = (b[i]-np.vdot(A[i,:],x)) / np.vdot(A[i,:],d)
                alphas_set.append(alph)
        alpha_bar = min(alphas_set)
        phi = lambda alpha : f(x + alpha * d)
        opt_alpha = minimize_scalar(phi, bounds=(0, alpha_bar), method='bounded').x
        #print("optimum alpha: ",opt_alpha)
        # increment x
        x = x + opt_alpha * d
        #print("x :", x)
        if np.vdot(Gradient(f)(x),d)>0 :
            break
    return x
        

In [16]:
A = np.array([
    [-1, 1],
    [ 1, 1],
    [ 0,-1]
],dtype = float)

b = np.array([7,5,-2])
x0 = np.array([-2,3])
x = feasible_directions(f,A,b,x0)
print(x)

1
1
direction  [ 1. -0.]
1
direction  [ 1. -0.]
[3.49719188e-06 2.00000596e+00]
